In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
sns.set()

In [2]:
user_ratings=pd.read_csv('user_ratings2.csv')
movies=pd.read_csv('movies3.csv')

In [3]:
user_ratings.head()

,userId,5,11,12,13,14,16,18,19,20,...,64499,69640,74510,77866,89492,98369,103731,114635,115210,116977
0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
len(user_ratings)

670

In [5]:
rows=[]
for index,row in user_ratings.iterrows():
    rows.append([index,row['userId']])
rowsnp=np.array(rows)
userid_index_mapping=pd.DataFrame(data=rowsnp,columns=['index','user_id'])

In [7]:
userid_index_mapping['index']=userid_index_mapping['index'].astype('int64')
userid_index_mapping['user_id']=userid_index_mapping['user_id'].astype('int64')

In [9]:
userid_index_mapping

,index,user_id
0,0,1
1,1,2
2,2,3
3,3,4
4,4,5
...,...,...
665,665,667
666,666,668
667,667,669
668,668,670


In [10]:
user_ratings_uid=user_ratings.set_index('userId')

In [11]:
movies_ids=[int(i) for i in user_ratings_uid.columns]
rows=[]
for i in range(len(movies_ids)):
    rows.append([i,movies_ids[i]])
movieid_index_mapping=pd.DataFrame(data=rows,columns=['index','movieId'])

In [12]:
movieid_index_mapping

,index,movieId
0,0,5
1,1,11
2,2,12
3,3,13
4,4,14
...,...,...
852,852,98369
853,853,103731
854,854,114635
855,855,115210


# Content Based Recommendation

## (1) Overview based

In [13]:
user_ratings_uid_copy=user_ratings_uid.copy()

In [14]:
movies.head()

,Unnamed: 0,budget,genres,id,overview,popularity,runtime,original_language,title
0,1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",285,"Captain Barbossa, long believed to be dead, ha...",139.082615,169.0,en,Pirates of the Caribbean: At World's End
1,5,258000000,"[{""id"": 14, ""name"": ""Fantasy""}, {""id"": 28, ""na...",559,The seemingly invincible Spider-Man goes up ag...,115.699814,139.0,en,Spider-Man 3
2,8,250000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",767,"As Harry begins his sixth year at Hogwarts, he...",98.885637,153.0,en,Harry Potter and the Half-Blood Prince
3,12,200000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",58,Captain Jack Sparrow works his way out of a bl...,145.847379,151.0,en,Pirates of the Caribbean: Dead Man's Chest
4,15,225000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 10751...",2454,One year after their incredible adventures in ...,53.978602,150.0,en,The Chronicles of Narnia: Prince Caspian


In [15]:
movies_copy=movies.copy()

In [16]:
overviews=movies['overview']

In [323]:
from sklearn.feature_extraction.text import TfidfVectorizer as tfidf
vectorizer=tfidf(stop_words='english',max_df=0.99,min_df=0.01)
overview_matrix=vectorizer.fit_transform(overviews)


In [324]:
overview_matrix.shape

(857, 415)

In [325]:
from sklearn.metrics.pairwise import sigmoid_kernel
sig=sigmoid_kernel(overview_matrix,overview_matrix)


In [326]:
sig[0]

array([0.76260429, 0.76159416, 0.76159416, 0.76178213, 0.76164531,
       0.76168086, 0.76159416, 0.76159416, 0.76162177, 0.76159416,
       0.76168184, 0.76159416, 0.76176188, 0.76159416, 0.76159416,
       0.76159416, 0.76159416, 0.76171401, 0.76166747, 0.76171567,
       0.76159416, 0.76159416, 0.76159416, 0.76159416, 0.76159416,
       0.76159416, 0.76159416, 0.76165721, 0.76167503, 0.76159416,
       0.76159416, 0.76168318, 0.76181568, 0.76159416, 0.76164507,
       0.76159416, 0.76159416, 0.76169725, 0.76169136, 0.76159416,
       0.76159416, 0.76159416, 0.76163094, 0.76159416, 0.76159416,
       0.76159416, 0.76159416, 0.76159416, 0.76159416, 0.76159416,
       0.76165154, 0.7617375 , 0.7616474 , 0.76159416, 0.76164946,
       0.76159416, 0.76159416, 0.76159416, 0.76159416, 0.76159416,
       0.76159416, 0.76167754, 0.76159416, 0.76159416, 0.76171965,
       0.76159416, 0.76159416, 0.76159416, 0.76159416, 0.76159416,
       0.76159416, 0.76159416, 0.76159416, 0.76159416, 0.76159

In [327]:
movies['title'].unique()

array(["Pirates of the Caribbean: At World's End", 'Spider-Man 3',
       'Harry Potter and the Half-Blood Prince',
       "Pirates of the Caribbean: Dead Man's Chest",
       'The Chronicles of Narnia: Prince Caspian',
       'Pirates of the Caribbean: On Stranger Tides',
       'The Golden Compass', 'King Kong', 'Titanic', 'Spider-Man 2',
       'Transformers: Revenge of the Fallen', 'Cars 2',
       'Terminator Salvation',
       'Indiana Jones and the Kingdom of the Crystal Skull',
       'The Dark Knight', 'Iron Man', 'Wild Wild West',
       'The Mummy: Tomb of the Dragon Emperor', 'The Polar Express',
       'Terminator 3: Rise of the Machines', "The Sorcerer's Apprentice",
       'Shrek the Third', 'Transformers', 'Alexander',
       'Harry Potter and the Goblet of Fire',
       'Charlie and the Chocolate Factory', 'Ratatouille',
       'Batman Begins', 'X-Men Origins: Wolverine',
       'The Matrix Revolutions', 'Mission: Impossible III', 'Armageddon',
       'Men in Black II'

In [328]:
movies[movies.title=='Captain America: The First Avenger']['id'].tolist()

[1771]

In [329]:
movieid_index_mapping[movieid_index_mapping.movieId==1771]['index'].tolist()[0]

448

In [330]:
def give_overview_recommendations(movie_title,sig):
    movie_id=movies[movies.title==movie_title]['id'].tolist()[0]
    index=movieid_index_mapping[movieid_index_mapping.movieId==movie_id]['index'].tolist()[0]
    similarity=sorted(list(enumerate(sig[index])),key=lambda x:x[1],reverse=True)
    recommended_movie_indices=[i[0] for i in similarity]
    recommended_movies=[]
    for i in recommended_movie_indices[1:11
                                      ]:
        movie_id_rec=movieid_index_mapping[movieid_index_mapping.index==i]['movieId'].tolist()[0]
        movie_title_rec=movies[movies.id==movie_id_rec]['title'].tolist()[0]
        recommended_movies.append(movie_title_rec)
    return recommended_movies
    
    

In [331]:
give_overview_recommendations('Captain America: The First Avenger',sig)

['Open Range',
 'The Mummy Returns',
 'Hitman',
 'Dead Man Walking',
 'Casper',
 'The Wedding Date',
 'Southland Tales',
 'Open Season',
 'Out of Sight',
 'The Brothers Grimm']

## (2) Content Based Recommendations

In [25]:
movies=movies_copy

In [26]:
movies.head()

,Unnamed: 0,budget,genres,id,overview,popularity,runtime,original_language,title
0,1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",285,"Captain Barbossa, long believed to be dead, ha...",139.082615,169.0,en,Pirates of the Caribbean: At World's End
1,5,258000000,"[{""id"": 14, ""name"": ""Fantasy""}, {""id"": 28, ""na...",559,The seemingly invincible Spider-Man goes up ag...,115.699814,139.0,en,Spider-Man 3
2,8,250000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",767,"As Harry begins his sixth year at Hogwarts, he...",98.885637,153.0,en,Harry Potter and the Half-Blood Prince
3,12,200000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",58,Captain Jack Sparrow works his way out of a bl...,145.847379,151.0,en,Pirates of the Caribbean: Dead Man's Chest
4,15,225000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 10751...",2454,One year after their incredible adventures in ...,53.978602,150.0,en,The Chronicles of Narnia: Prince Caspian


In [27]:
movies=movies.iloc[:,1:]

In [28]:
movies.head()

,budget,genres,id,overview,popularity,runtime,original_language,title
0,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",285,"Captain Barbossa, long believed to be dead, ha...",139.082615,169.0,en,Pirates of the Caribbean: At World's End
1,258000000,"[{""id"": 14, ""name"": ""Fantasy""}, {""id"": 28, ""na...",559,The seemingly invincible Spider-Man goes up ag...,115.699814,139.0,en,Spider-Man 3
2,250000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",767,"As Harry begins his sixth year at Hogwarts, he...",98.885637,153.0,en,Harry Potter and the Half-Blood Prince
3,200000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",58,Captain Jack Sparrow works his way out of a bl...,145.847379,151.0,en,Pirates of the Caribbean: Dead Man's Chest
4,225000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 10751...",2454,One year after their incredible adventures in ...,53.978602,150.0,en,The Chronicles of Narnia: Prince Caspian


In [29]:
import json
movies['genres']=movies['genres'].apply(json.loads)

In [30]:
movies['genres']

0      [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...
1      [{'id': 14, 'name': 'Fantasy'}, {'id': 28, 'na...
2      [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...
3      [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...
4      [{'id': 12, 'name': 'Adventure'}, {'id': 10751...
                             ...                        
852    [{'id': 18, 'name': 'Drama'}, {'id': 10402, 'n...
853    [{'id': 99, 'name': 'Documentary'}, {'id': 36,...
854    [{'id': 9648, 'name': 'Mystery'}, {'id': 18, '...
855                       [{'id': 35, 'name': 'Comedy'}]
856    [{'id': 27, 'name': 'Horror'}, {'id': 35, 'nam...
Name: genres, Length: 857, dtype: object

In [31]:
def pipe_flatten_names(keywords):
    return '|'.join([x['name'] for x in keywords])
movies['genres'] = movies['genres'].apply(pipe_flatten_names)

In [32]:
movies['genres']

0      Adventure|Fantasy|Action
1      Fantasy|Action|Adventure
2      Adventure|Fantasy|Family
3      Adventure|Fantasy|Action
4      Adventure|Family|Fantasy
                 ...           
852         Drama|Music|Romance
853         Documentary|History
854      Mystery|Drama|Thriller
855                      Comedy
856         Horror|Comedy|Crime
Name: genres, Length: 857, dtype: object

In [33]:
cleaned = movies.genres.str.split('|', expand=True).stack()
dummies=pd.get_dummies(cleaned, prefix='g').groupby(level=0).sum()
movies_encoded=movies.merge(dummies,left_index=True, right_index=True)






In [34]:
movies_encoded.head()

,budget,genres,id,overview,popularity,runtime,original_language,title,g_Action,g_Adventure,...,g_Foreign,g_History,g_Horror,g_Music,g_Mystery,g_Romance,g_Science Fiction,g_Thriller,g_War,g_Western
0,300000000,Adventure|Fantasy|Action,285,"Captain Barbossa, long believed to be dead, ha...",139.082615,169.0,en,Pirates of the Caribbean: At World's End,1,1,...,0,0,0,0,0,0,0,0,0,0
1,258000000,Fantasy|Action|Adventure,559,The seemingly invincible Spider-Man goes up ag...,115.699814,139.0,en,Spider-Man 3,1,1,...,0,0,0,0,0,0,0,0,0,0
2,250000000,Adventure|Fantasy|Family,767,"As Harry begins his sixth year at Hogwarts, he...",98.885637,153.0,en,Harry Potter and the Half-Blood Prince,0,1,...,0,0,0,0,0,0,0,0,0,0
3,200000000,Adventure|Fantasy|Action,58,Captain Jack Sparrow works his way out of a bl...,145.847379,151.0,en,Pirates of the Caribbean: Dead Man's Chest,1,1,...,0,0,0,0,0,0,0,0,0,0
4,225000000,Adventure|Family|Fantasy,2454,One year after their incredible adventures in ...,53.978602,150.0,en,The Chronicles of Narnia: Prince Caspian,0,1,...,0,0,0,0,0,0,0,0,0,0


In [35]:
movies=movies_encoded.drop('genres',axis=1)

In [36]:
movies_copy10=movies.copy()

In [37]:
movies

,budget,id,overview,popularity,runtime,original_language,title,g_Action,g_Adventure,g_Animation,...,g_Foreign,g_History,g_Horror,g_Music,g_Mystery,g_Romance,g_Science Fiction,g_Thriller,g_War,g_Western
0,300000000,285,"Captain Barbossa, long believed to be dead, ha...",139.082615,169.0,en,Pirates of the Caribbean: At World's End,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,258000000,559,The seemingly invincible Spider-Man goes up ag...,115.699814,139.0,en,Spider-Man 3,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,250000000,767,"As Harry begins his sixth year at Hogwarts, he...",98.885637,153.0,en,Harry Potter and the Half-Blood Prince,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,200000000,58,Captain Jack Sparrow works his way out of a bl...,145.847379,151.0,en,Pirates of the Caribbean: Dead Man's Chest,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4,225000000,2454,One year after their incredible adventures in ...,53.978602,150.0,en,The Chronicles of Narnia: Prince Caspian,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
852,160000,5723,A vacuum repairman moonlights as a street musi...,19.052179,85.0,en,Once,0,0,0,...,0,0,0,1,0,1,0,0,0,0
853,160000,1779,A documentary about the closure of General Mot...,3.284903,91.0,en,Roger & Me,0,0,0,...,0,1,0,0,0,0,0,0,0,0
854,60000,473,The debut film from Darren Aronofsky in which ...,27.788067,84.0,en,Pi,0,0,0,...,0,0,0,0,1,0,0,1,0,0
855,27000,2292,Convenience and video store clerks Dante and R...,19.748658,92.0,en,Clerks,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
overview_tfidf=overview_matrix

In [39]:
overview_cols=vectorizer.get_feature_names()

In [40]:
from pandas import DataFrame

In [41]:
overview_df=pd.DataFrame(overview_tfidf.toarray(),columns=overview_cols)

In [42]:
movies_with_overview=pd.concat([movies,overview_df],axis=1)

In [43]:
movies_with_overview

,budget,id,overview,popularity,runtime,original_language,title,g_Action,g_Adventure,g_Animation,...,works,world,worse,writer,wrong,year,years,york,young,younger
0,300000000,285,"Captain Barbossa, long believed to be dead, ha...",139.082615,169.0,en,Pirates of the Caribbean: At World's End,1,1,0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
1,258000000,559,The seemingly invincible Spider-Man goes up ag...,115.699814,139.0,en,Spider-Man 3,1,1,0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
2,250000000,767,"As Harry begins his sixth year at Hogwarts, he...",98.885637,153.0,en,Harry Potter and the Half-Blood Prince,0,1,0,...,0.000000,0.0,0.0,0.0,0.0,0.207012,0.0,0.0,0.000000,0.0
3,200000000,58,Captain Jack Sparrow works his way out of a bl...,145.847379,151.0,en,Pirates of the Caribbean: Dead Man's Chest,1,1,0,...,0.388641,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
4,225000000,2454,One year after their incredible adventures in ...,53.978602,150.0,en,The Chronicles of Narnia: Prince Caspian,0,1,0,...,0.000000,0.0,0.0,0.0,0.0,0.184659,0.0,0.0,0.148494,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
852,160000,5723,A vacuum repairman moonlights as a street musi...,19.052179,85.0,en,Once,0,0,0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
853,160000,1779,A documentary about the closure of General Mot...,3.284903,91.0,en,Roger & Me,0,0,0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
854,60000,473,The debut film from Darren Aronofsky in which ...,27.788067,84.0,en,Pi,0,0,0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
855,27000,2292,Convenience and video store clerks Dante and R...,19.748658,92.0,en,Clerks,0,0,0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0


In [44]:
movies_with_overview=movies_with_overview.drop('overview',axis=1)

In [45]:
movies_with_overview.columns


Index(['budget', 'id', 'popularity', 'runtime', 'original_language', 'title',
       'g_Action', 'g_Adventure', 'g_Animation', 'g_Comedy',
       ...
       'works', 'world', 'worse', 'writer', 'wrong', 'year', 'years', 'york',
       'young', 'younger'],
      dtype='object', length=1042)

In [46]:
from sklearn.preprocessing import MinMaxScaler

In [47]:
user_ratings_nonan=user_ratings.fillna(int(0))

In [48]:
user_ratings_nonan

,userId,5,11,12,13,14,16,18,19,20,...,64499,69640,74510,77866,89492,98369,103731,114635,115210,116977
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
665,667,0.0,3.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
666,668,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
667,669,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
668,670,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [49]:
! pip install --upgrade pandas

Requirement already up-to-date: pandas in c:\users\admin\new folder\lib\site-packages (1.0.5)


You should consider upgrading via the 'c:\users\admin\new folder\python.exe -m pip install --upgrade pip' command.


In [50]:
ratings_matrix=user_ratings_nonan.iloc[:,1:].values


In [51]:
ratings_matrix.shape

(670, 857)

In [52]:
movies_with_overview.head()

,budget,id,popularity,runtime,original_language,title,g_Action,g_Adventure,g_Animation,g_Comedy,...,works,world,worse,writer,wrong,year,years,york,young,younger
0,300000000,285,139.082615,169.0,en,Pirates of the Caribbean: At World's End,1,1,0,0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
1,258000000,559,115.699814,139.0,en,Spider-Man 3,1,1,0,0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
2,250000000,767,98.885637,153.0,en,Harry Potter and the Half-Blood Prince,0,1,0,0,...,0.000000,0.0,0.0,0.0,0.0,0.207012,0.0,0.0,0.000000,0.0
3,200000000,58,145.847379,151.0,en,Pirates of the Caribbean: Dead Man's Chest,1,1,0,0,...,0.388641,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
4,225000000,2454,53.978602,150.0,en,The Chronicles of Narnia: Prince Caspian,0,1,0,0,...,0.000000,0.0,0.0,0.0,0.0,0.184659,0.0,0.0,0.148494,0.0


In [53]:
movies_encoded=pd.get_dummies(data=movies_with_overview, columns=['original_language'])

In [54]:
movies_copy50=movies_encoded.copy()

In [55]:
movies_encoded

,budget,id,popularity,runtime,title,g_Action,g_Adventure,g_Animation,g_Comedy,g_Crime,...,original_language_it,original_language_ja,original_language_ko,original_language_nl,original_language_pt,original_language_ro,original_language_ru,original_language_sv,original_language_th,original_language_zh
0,300000000,285,139.082615,169.0,Pirates of the Caribbean: At World's End,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,258000000,559,115.699814,139.0,Spider-Man 3,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,250000000,767,98.885637,153.0,Harry Potter and the Half-Blood Prince,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,200000000,58,145.847379,151.0,Pirates of the Caribbean: Dead Man's Chest,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,225000000,2454,53.978602,150.0,The Chronicles of Narnia: Prince Caspian,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
852,160000,5723,19.052179,85.0,Once,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
853,160000,1779,3.284903,91.0,Roger & Me,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
854,60000,473,27.788067,84.0,Pi,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
855,27000,2292,19.748658,92.0,Clerks,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [56]:
scaler = MinMaxScaler()
movies_matrix=movies_encoded.drop(['title','id'],axis=1)
movies_matrix_scaled=scaler.fit_transform(movies_matrix)

In [57]:
movies_matrix_scaled.shape

(857, 1055)

In [58]:
# movies_matrix=movies_encoded.drop(['title','id'],axis=1)

In [59]:
# movies_matrix_encoded=movies_matrix_scaled.values

In [60]:
movies_matrix_scaled.shape

(857, 1055)

In [61]:
user_preferences=np.dot(ratings_matrix,movies_matrix_scaled)

In [62]:
row_sums = user_preferences.sum(axis=1)
user_preferences = user_preferences / row_sums[:, np.newaxis]

In [75]:
user_preferences.shape

(670, 1055)

In [76]:
weighted_features_matrix=np.dot(user_preferences,movies_matrix_scaled.T)

In [77]:
weighted_features_matrix.shape

(670, 857)

In [78]:
ratings_matrix.shape

(670, 857)

In [79]:
req_matrix=np.where(ratings_matrix==0,weighted_features_matrix,-np.inf)

In [317]:
def give_content_recommendations(user_id,req_matrix):
#     movie_id=movies[user_.title==movie_title]['id'].tolist()[0]
    index=userid_index_mapping[userid_index_mapping.user_id==user_id]['index'].tolist()[0]
#     matrix_req=weighted_features_matrix*tarings_matrix
    movies_rec=sorted(list(enumerate(req_matrix[index])),key=lambda x:x[1],reverse=True)
    recommended_movie_indices=[i[0] for i in movies_rec]
    recommended_movies=[]
    for i in recommended_movie_indices[0:11]:
        movie_id_rec=movieid_index_mapping[movieid_index_mapping.index==i]['movieId'].tolist()[0]
        movie_title_rec=movies[movies.id==movie_id_rec]['title'].tolist()[0]
        recommended_movies.append(movie_title_rec)
    return recommended_movies

In [81]:
userid_index_mapping

,index,user_id
0,0,1
1,1,2
2,2,3
3,3,4
4,4,5
...,...,...
665,665,667
666,666,668
667,667,669
668,668,670


In [82]:
give_overview_recommendations(5,req_matrix)

['Bowling for Columbine',
 'The Glass House',
 'The Mummy Returns',
 'Saw II',
 'Eternal Sunshine of the Spotless Mind',
 'Taxi Driver',
 'Lethal Weapon 4',
 'The 13th Warrior',
 "A Hard Day's Night",
 'Indiana Jones and the Last Crusade',
 'Captain America: The First Avenger']

# Content Based Recommendations are made

# Collaborative Filtering

## (1) Collaborative Filtering

In [83]:
user_ratings

,userId,5,11,12,13,14,16,18,19,20,...,64499,69640,74510,77866,89492,98369,103731,114635,115210,116977
0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
665,667,NaN,3.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
666,668,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
667,669,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
668,670,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [84]:
user_ratings_knn=user_ratings.set_index('userId')

In [85]:
user_ratings_knn

,5,11,12,13,14,16,18,19,20,22,...,64499,69640,74510,77866,89492,98369,103731,114635,115210,116977
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,NaN,3.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
668,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
669,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [86]:
from sklearn.model_selection import GridSearchCV


In [311]:
user_ratings_no_nan=user_ratings.set_index('userId').fillna(0)
user_ratings_nonan_matrix=user_ratings_no_nan.values
user_ratings_nonan_matrix=np.array(user_ratings_nonan_matrix)

In [312]:
row_sums=np.sum(user_ratings_nonan_matrix,axis=1)
user_ratings_nonan_matrix_normalized=user_ratings_nonan_matrix-row_sums[:,np.newaxis]



In [301]:
# def make_matrices(dimensions):
#     user_embedding=np.random.rand(len(user_ratings_no_nan),dimensions)
#     movie_embedding=np.random.rand(len(user_ratings_no_nan.columns),dimensions)
#     return user_embedding,movie_embedding

In [302]:
# dimensions=3

In [303]:
# user_embedding,movie_embedding=make_matrices(dimensions)

In [304]:
# user_embedding.shape
# movie_embedding.shape

(670, 3)

In [308]:
# for i in range(len(user_ratings_nonan_matrix)):
#     for j in range(len(user_ratings_nonan_matrix[i])):
#         user_ratings_nonan_matrix_normalized[i][j]=int(user_ratings_nonan_matrix_normalized[i][j])

In [297]:
# def check(a):
#     if len(costs)>20:
#         for i in range(len(costs)-1,len(costs)-1-20,-1):
#             if int(a[i])!=int(a[i-1]):
#                 return True
#         return False
#     return True

In [294]:
# def get_embeddings(user_embedding,movie_embedding,user_ratings,dimensions,alpha,beta,steps):
#     movie_embedding=movie_embedding.T
# #     matrix=np.dot(user_embedding,movie_embedding)
# #     user_ratings_nonan=np.where(user_ratings==np.nan,0,user_ratings-matrix)
#     cost=0
# #     steps=100
#     costs=[]
#     for step in range(steps):
#         for i in range(len(user_ratings)):
#             for j in range(len(user_ratings[i])):
#                 if user_ratings[i][j]>0:
#                     costij=user_ratings[i][j]-np.dot(user_embedding[i,:],movie_embedding[:,j])
#                     for k in range(dimensions):
#                         user_embedding[i][k] = user_embedding[i][k] + alpha * (2 * costij * movie_embedding[k][j] - beta * user_embedding[i][k])
#                         movie_embedding[k][j] = movie_embedding[k][j] + alpha * (2 * costij * user_embedding[i][k] - beta * movie_embedding[k][j])
#         cost=0
#         matrix=np.dot(user_embedding,movie_embedding)
#         for i in range(len(user_ratings)):
#             for j in range(len(user_ratings[i])):
#                 if user_ratings[i][j]>0:
#                     cost+=pow(user_ratings[i][j]-np.dot(user_embedding[i,:],movie_embedding[:,j]),2)
#                     for k in range(dimensions):
#                         cost+=(beta/2)*(pow(user_embedding[i][k],2)+pow(movie_embedding[k][j],2))
#         if cost<0.01:
#             break
        
#     return user_embedding,movie_embedding.T,cost
        
        
    
    
    

In [281]:
# user_embedding_copy=user_embedding
# movie_embedding_copy=movie_embedding

In [295]:
# user_embedding=user_embedding_copy
# movie_embedding=movie_embedding_copy

In [296]:
# user_embedding,movie_embedding,rmse=get_embeddings(user_embedding,movie_embedding,user_ratings_nonan_matrix,dimensions,alpha=0.0005,beta=0.02,steps=4000)

0 5669.1591678735695
1 5669.051594404126
2 5668.9626366120765
3 5668.875209692672
4 5668.78804072718
5 5668.701450495168
6 5668.615617470005
7 5668.530542191675
8 5668.446149629165
9 5668.362348652611
10 5668.279055816616
11 5668.1962018769755
12 5668.1137315233045
13 5668.031601039357
14 5667.949775731673
15 5667.868227734829
16 5667.7869343037455
17 5667.70587654501
18 5667.625038491101
19 5667.544406431222
20 5667.4639684252215


NameError: name 'a' is not defined

In [335]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(user_ratings_nonan_matrix_normalized, k = 50)
sigma = np.diag(sigma)
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + row_sums.reshape(-1, 1)
# preds_df = pd.DataFrame(all_user_predicted_ratings, columns = R_df.columns)

In [336]:
#You might like
def give_colab_recommendations(user_id):
#     movie_id=movies[user_.title==movie_title]['id'].tolist()[0]
    index=userid_index_mapping[userid_index_mapping.user_id==user_id]['index'].tolist()[0]
#     matrix_req=weighted_features_matrix*tarings_matrix
    movies_rec=sorted(list(enumerate(all_user_predicted_ratings[index])),key=lambda x:x[1],reverse=True)
    recommended_movie_indices=[i[0] for i in movies_rec]
    recommended_movies=[]
    for i in recommended_movie_indices[0:11]:
        movie_id_rec=movieid_index_mapping[movieid_index_mapping.index==i]['movieId'].tolist()[0]
        movie_title_rec=movies[movies.id==movie_id_rec]['title'].tolist()[0]
        recommended_movies.append(movie_title_rec)
    return recommended_movies



In [337]:
give_colab_recommendations(1)

['American Pie',
 'Barry Lyndon',
 'Reign Over Me',
 'Jay and Silent Bob Strike Back',
 'Jurassic Park',
 'Kiss of the Dragon',
 'Closer',
 'Young Frankenstein',
 'Cold Mountain',
 "Mr. Holland's Opus",
 'Stranger Than Fiction']

# KNN Recommendations

In [338]:
user_ratings.head()

,userId,5,11,12,13,14,16,18,19,20,...,64499,69640,74510,77866,89492,98369,103731,114635,115210,116977
0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [339]:
user_ratings_idindexed=user_ratings.set_index('userId')

In [344]:
user_ratings_idindexed.head()
user_ratings_idindexed=user_ratings_idindexed.fillna(0)

In [345]:
movies_ratings=user_ratings_idindexed.T

In [346]:
movies_ratings.head()

userId,1,2,3,4,5,6,7,8,9,10,...,662,663,664,665,666,667,668,669,670,671
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [347]:
from sklearn.neighbors import NearestNeighbors
knn_model=NearestNeighbors(metric='cosine',algorithm='brute')
knn_model.fit(movies_ratings)


NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [362]:
def nn_recommendations(movie_title):
    movie_id=movies[movies.title==movie_title]['id'].tolist()[0]
    index=movieid_index_mapping[movieid_index_mapping.movieId==movie_id]['index'].tolist()[0]
#     similarity=sorted(list(enumerate(sig[index])),key=lambda x:x[1],reverse=True)
    distances,indices=knn_model.kneighbors(movies_ratings.iloc[index,:].values.reshape(1,-1),n_neighbors=11)
#     print(indices)
    recommended_movies=[]
    for i in indices[0][1:]:
        movie_id_rec=movieid_index_mapping[movieid_index_mapping.index==i]['movieId'].tolist()[0]
        movie_title_rec=movies[movies.id==movie_id_rec]['title'].tolist()[0]
        recommended_movies.append(movie_title_rec)
    return recommended_movies
    

In [363]:
nn_recommendations('Captain America: The First Avenger')

['The Empire Strikes Back',
 'Open Range',
 'Ladyhawke',
 'Pieces of April',
 'Miss Congeniality',
 'My Blueberry Nights',
 '1408',
 "The Devil's Rejects",
 'What Lies Beneath',
 'Groundhog Day']